### Importando bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl as px
from datetime import datetime
import os

### Lendo a planilha

In [37]:
df_path = ('/Users/jhenyfferborges/Documents/DSW_analysis/projeto_dsw/data/Weekly DSW order report 2025-05-04 without PI.xlsx')
df = pd.read_excel(df_path, sheet_name="Main Extract")
df_backup = df.copy()

In [ ]:
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.width', 1000)  # Ajuste a largura da exibição
df

### Nessa parte resolvi verificar o tamanho do dataset, linhas e colunas, temos 50mil linhas e 93 (tinnham 102 inicialmente, mas decidi mudar meu excel, pois os dados estavam muito estranhos) colunas. Essa quantidade é referente a todas as transações realizadas pelo Ecommerce mundialmente, a seguir irei aplicar um filtro para analisarmos apenas Brasil e Mexico

In [38]:
df_la = df[df['Sold To Country Code Description'].isin(['Brazil', 'Mexico'])]

# Visualizar as primeiras linhas do novo DataFrame
df_la.head()

Provisioning Hold Release Date Sold To IBM Customer Number  Sold To Customer Number          Sold To Customer Name Full Sold To Country Code Description Geography  SAP Sales Order Number Distribution Channel Alternate Software Brand Code Dscr IBM Software Pillar Code Part Number             Revenue Stream Code Dscr IBM Software Pillar Code.1 Software Product Brand Code Dscr                           Part Description Display  Part Quantity  Coverage Term Start Date   End Date  SaaS Annual Commitment Value Local  SaaS Total Commitment Value Local Currency Code  SaaS Annual Commitment Value USD  SaaS Total Commitment Value USD  Renewal Counter         SaaS Renewal Model Code Dscr Provisioning Hold Release Date.1  Cancel Reason Code Sold To IBM Domestic Buying Group Number Sold To IBM Domestic Client Number Portal Source Indicator Sold To IBM Worldwide ISU Code Description Sold To Coverage Group Id  Sold To Coverage Base Number           Sold To Coverage Base Name  \
62         2024-01-05 10:14:16.000                      002603                  4340956           BRASIL AGRO SERVICES LTDA                           Brazil  AMERICAS                77550823                    C                           Security                    35ADY     D1X6SLL                      SaaS ELOS Setup                      35ADY                         Security          IBM MaaS360 Customer Setup per Engagement              1             12 2024-01-03 2025-01-02                             4539.21                            4539.21           BRL                          864.6114                         864.6114                0                                  NaN          2024-01-05 10:14:16.000                 NaN                                 UNASSIGN                           DC01H7TV                     SCW   Cross Industry (for Accounting use only)                       758                       12773.0                  BR - DSS Sul Retail   
63         2024-01-05 10:14:16.000                      002603                  4340956           BRASIL AGRO SERVICES LTDA                           Brazil  AMERICAS                77550823                    C                           Security                    35ADY     D1P3GLL  SaaS Sbscrptn Mnthly or Annl w/Supt                      35ADY                         Security  IBM MaaS360 Essentials Suite (SaaS) Managed Cl...             20             12 2024-01-03 2025-01-02                             5968.80                            5968.80           BRL                         1136.9434                        1136.9434                1  Rnw srvc trm orgnl trm (chng allwd)          2024-01-05 10:14:16.000                 NaN                                 UNASSIGN                           DC01H7TV                     SCW   Cross Industry (for Accounting use only)                       758                       12773.0                  BR - DSS Sul Retail   
225        2024-01-10 16:28:13.999                     UNKNOWN                  4339775          Elda Frinne Galicia Moreno                           Mexico  AMERICAS                77551318                    C                          Data & AI                    35A0V     D1QWVLL  SaaS Sbscrptn Mnthly or Annl w/Supt                      35A0V             Analytics - Platform  IBM SPSS Statistics Subscription, Base Edition...              1              1 2024-01-10 2024-02-09                              100.27                             100.27           USD                          100.2700                         100.2700                0  Rnw srvc trm orgnl trm (chng allwd)          2024-01-10 16:28:13.999                98.0                                 UNASSIGN                           UNASSIGN                     SCW                             To Be Assigned                   UNKNOWN                           NaN                                  NaN   
273        2024-01-15 11:29:29.000                      002395              

In [39]:
num_linhas, num_colunas = df_la.shape
print(f"Quantidade de linhas: {num_linhas}")
print(f"Quantidade de colunas: {num_colunas}")

Quantidade de linhas: 371
Quantidade de colunas: 86


In [40]:
# Ajustar as configurações de exibição do pandas para mostrar todas as colunas
pd.set_option('display.max_columns', None)

# obtendo o tipo de dados das colunas
data_types = df_la.dtypes

# obtendo a quantidade de valores nulos em cada coluna
null_values = df_la.isnull().sum()

#Combinand informações
info_colunas = pd.DataFrame({'Tipo de dados': data_types, 'Valores Nulos': null_values})

#visualizando as informações combinadas
print(info_colunas.to_string())

                                             Tipo de dados  Valores Nulos
Provisioning Hold Release Date              datetime64[ns]             17
Sold To IBM Customer Number                         object              0
Sold To Customer Number                              int64              0
Sold To Customer Name Full                          object              0
Sold To Country Code Description                    object              0
Geography                                           object              0
SAP Sales Order Number                               int64              0
Distribution Channel                                object              0
Alternate Software Brand Code Dscr                  object              0
IBM Software Pillar Code                            object              0
Part Number                                         object              0
Revenue Stream Code Dscr                            object              0
IBM Software Pillar Code.1            

### Abaixo, eu exclui algumas colunas que possuem apenas valores nulos, se no futuro esses valores vierem a ser preenchidos, então poderei deixa-los, se necessário.

In [41]:
columns_exclude = ['Part Number',
                    'Currency Code',
                    'Back Dated Start Date',
                    'Add Reason Code',
                    'Quote Number',
                    'Legacy Opportunity Number',
                    'Opportunity Number',
                    'Sales Rep Id',
                    'Last Billing Date',
                     ]

df_columns_excluded = df_la.drop(columns=columns_exclude)
df_columns_excluded.head()

Provisioning Hold Release Date Sold To IBM Customer Number  Sold To Customer Number          Sold To Customer Name Full Sold To Country Code Description Geography  SAP Sales Order Number Distribution Channel Alternate Software Brand Code Dscr IBM Software Pillar Code             Revenue Stream Code Dscr IBM Software Pillar Code.1 Software Product Brand Code Dscr                           Part Description Display  Part Quantity  Coverage Term Start Date   End Date  SaaS Annual Commitment Value Local  SaaS Total Commitment Value Local  SaaS Annual Commitment Value USD  SaaS Total Commitment Value USD  Renewal Counter         SaaS Renewal Model Code Dscr Provisioning Hold Release Date.1  Cancel Reason Code Sold To IBM Domestic Buying Group Number Sold To IBM Domestic Client Number Portal Source Indicator Sold To IBM Worldwide ISU Code Description Sold To Coverage Group Id  Sold To Coverage Base Number           Sold To Coverage Base Name Sold To Coverage Number  \
62         2024-01-05 10:14:16.000                      002603                  4340956           BRASIL AGRO SERVICES LTDA                           Brazil  AMERICAS                77550823                    C                           Security                    35ADY                      SaaS ELOS Setup                      35ADY                         Security          IBM MaaS360 Customer Setup per Engagement              1             12 2024-01-03 2025-01-02                             4539.21                            4539.21                          864.6114                         864.6114                0                                  NaN          2024-01-05 10:14:16.000                 NaN                                 UNASSIGN                           DC01H7TV                     SCW   Cross Industry (for Accounting use only)                       758                       12773.0                  BR - DSS Sul Retail                 0012773   
63         2024-01-05 10:14:16.000                      002603                  4340956           BRASIL AGRO SERVICES LTDA                           Brazil  AMERICAS                77550823                    C                           Security                    35ADY  SaaS Sbscrptn Mnthly or Annl w/Supt                      35ADY                         Security  IBM MaaS360 Essentials Suite (SaaS) Managed Cl...             20             12 2024-01-03 2025-01-02                             5968.80                            5968.80                         1136.9434                        1136.9434                1  Rnw srvc trm orgnl trm (chng allwd)          2024-01-05 10:14:16.000                 NaN                                 UNASSIGN                           DC01H7TV                     SCW   Cross Industry (for Accounting use only)                       758                       12773.0                  BR - DSS Sul Retail                 0012773   
225        2024-01-10 16:28:13.999                     UNKNOWN                  4339775          Elda Frinne Galicia Moreno                           Mexico  AMERICAS                77551318                    C                          Data & AI                    35A0V  SaaS Sbscrptn Mnthly or Annl w/Supt                      35A0V             Analytics - Platform  IBM SPSS Statistics Subscription, Base Edition...              1              1 2024-01-10 2024-02-09                              100.27                             100.27                          100.2700                         100.2700                0  Rnw srvc trm orgnl trm (chng allwd)          2024-01-10 16:28:13.999                98.0                                 UNASSIGN                           UNASSIGN                     SCW                             To Be Assigned                   UNKNOWN                           NaN                                  NaN                 UNKNOWN   
273        2024-01-15 11:29:29.000                      002395                  4341

In [ ]:
#### Esperar pra rodar esse e ver se vai precisar
#df_la['Start Date'] = df_la['Start Date'].fillna(method='ffill')
#df_la['End Date'] = df_la['End Date'].fillna(method='ffill')

# Não estou rodando o bloco abaixo atualmente pois as colunas já estão setadas como datetime e as vezes que rodei esse bloco as colunas se tornam object. 

In [ ]:
# Lista de colunas que precisam ser convertidas para datetime
colunas_para_datetime = [
    'Provisioning Hold Release Date', 
    'Start Date', 
    'End Date',
    'Provisioning Hold Release Date.1',
    'Sales Order Date',
    'Firm Order Date',
    'Sales Order Creation Date',
    'Back Dated Start Date',
    'Add Date Header', 
    'Add Date Line Item', 
    'SaaS Terminated Date'
]

# Função para converter as colunas e remover a parte de hora
def converter_e_formatar_col(dataframe, coluna):
    try:
        # Converte para datetime, sem hora, com erro 'coerce' para valores inválidos
        dataframe[coluna] = pd.to_datetime(dataframe[coluna], errors='coerce').dt.date
        # Verifica se a conversão foi bem-sucedida
        if pd.api.types.is_object_dtype(dataframe[coluna]):
            print(f"A coluna {coluna} não foi convertida corretamente.")
    except Exception as e:
        print(f"Erro ao tentar converter a coluna {coluna}: {e}")

# Aplicar a conversão para cada coluna na lista
for coluna in colunas_para_datetime:
    if coluna in df_la.columns:
        converter_e_formatar_col(df_la, coluna)

# Visualizar as primeiras linhas para conferir a conversão
print(df_la[['Sold To Customer Name Full', 'Start Date', 'End Date', 'Provisioning Hold Release Date']].head())







In [34]:
# Remover a parte de hora da coluna 'Provisioning Hold Release Date'
df_la['Provisioning Hold Release Date'] = df_la['Provisioning Hold Release Date'].dt.date

# Visualizar as primeiras linhas para verificar
print(df_la[['Sold To Customer Name Full', 'Provisioning Hold Release Date']].head())


             Sold To Customer Name Full Provisioning Hold Release Date
62            BRASIL AGRO SERVICES LTDA                     2024-01-05
63            BRASIL AGRO SERVICES LTDA                     2024-01-05
225          Elda Frinne Galicia Moreno                     2024-01-10
273     PSEG CENTRO DE PESQUISA CLINICA                     2024-01-15
382  LUMIO MARKETING E TEC DA INFO LTDA                     2024-01-17


/var/folders/fy/nyylvm7155592lqbprq96mm80000gn/T/ipykernel_30087/3784673745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_la['Provisioning Hold Release Date'] = df_la['Provisioning Hold Release Date'].dt.date


In [43]:
print(info_colunas.to_string())

                                             Tipo de dados  Valores Nulos
Provisioning Hold Release Date              datetime64[ns]             17
Sold To IBM Customer Number                         object              0
Sold To Customer Number                              int64              0
Sold To Customer Name Full                          object              0
Sold To Country Code Description                    object              0
Geography                                           object              0
SAP Sales Order Number                               int64              0
Distribution Channel                                object              0
Alternate Software Brand Code Dscr                  object              0
IBM Software Pillar Code                            object              0
Part Number                                         object              0
Revenue Stream Code Dscr                            object              0
IBM Software Pillar Code.1            

In [44]:
# Lista de colunas que precisam ser convertidas para numérico
colunas_para_numerico = [
    'Sold To IBM Customer Number', 
    'Sold To Coverage Number', 
    'Order Confirmation Number',
    'SAP Sales Order Number',
    'Part Number',
    'Coverage Term',
    'Currency Code',
    'Cancel Reason Code',
    'Sold To Coverage Base Number'
]

# Aplicar a conversão para cada coluna na lista

df_la[colunas_para_numerico] = df_la[colunas_para_numerico].apply(pd.to_numeric, errors='coerce').astype('Int64')


/var/folders/fy/nyylvm7155592lqbprq96mm80000gn/T/ipykernel_30087/2637870682.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_la[colunas_para_numerico] = df_la[colunas_para_numerico].apply(pd.to_numeric, errors='coerce').astype('Int64')


### Abaixo vou preencher os campos nulos com alguns valores

In [25]:
# preenchendo valores nulos
df_la['Cancel Reason Code'] = df_la['Cancel Reason Code'].fillna(90)
df_la['Cancel Reason Dscr'] = df_la['Cancel Reason Dscr'].fillna('No cancellation')

/var/folders/fy/nyylvm7155592lqbprq96mm80000gn/T/ipykernel_86295/2949828604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_la['Cancel Reason Code'] = df_la['Cancel Reason Code'].fillna(90)
/var/folders/fy/nyylvm7155592lqbprq96mm80000gn/T/ipykernel_86295/2949828604.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_la['Cancel Reason Dscr'] = df_la['Cancel Reason Dscr'].fillna('No cancellation')


In [45]:
df_la.info()
df_la.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 371 entries, 62 to 16418
Data columns (total 86 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Provisioning Hold Release Date              354 non-null    datetime64[ns]
 1   Sold To IBM Customer Number                 267 non-null    Int64         
 2   Sold To Customer Number                     371 non-null    int64         
 3   Sold To Customer Name Full                  371 non-null    object        
 4   Sold To Country Code Description            371 non-null    object        
 5   Geography                                   371 non-null    object        
 6   SAP Sales Order Number                      371 non-null    Int64         
 7   Distribution Channel                        371 non-null    object        
 8   Alternate Software Brand Code Dscr          371 non-null    object        
 9   IBM Software

Provisioning Hold Release Date  Sold To IBM Customer Number  Sold To Customer Number  SAP Sales Order Number  Part Number  Part Quantity  Coverage Term                     Start Date                       End Date  SaaS Annual Commitment Value Local  SaaS Total Commitment Value Local  Currency Code  SaaS Annual Commitment Value USD  SaaS Total Commitment Value USD  Renewal Counter Provisioning Hold Release Date.1  Cancel Reason Code  Sold To Coverage Base Number  Sold To Coverage Number               Sales Order Date                Firm Order Date      Sales Order Creation Date  Back Dated Start Date                Add Date Header             Add Date Line Item  Cumulative Coverage Term Last Billing Date  Line Item Seq Number  Local Amount to Be Billed  Local Extended Price  Local Unit Price  Order Confirmation Number  Ramp Up Flag  SaaS Billing Order Day of the Month           SaaS Terminated Date  Sales Rep Id  USD Extended Price  USD Unit Price  Opportunity Number  \
count                            354                        267.0             3.710000e+02                   371.0          0.0     371.000000          371.0                            352                            352                          371.000000                         371.000000            0.0                        371.000000                       371.000000       371.000000                              354               185.0                         267.0                    267.0                            371                            360                            371                    0.0                            371                            371                     371.0                 0            371.000000                      371.0            371.000000        371.000000                      354.0         371.0                           371.000000                            187           0.0          371.000000      371.000000                 0.0   
mean   2024-09-11 16:41:26.380932352                148631.082397             4.370250e+06         77573905.231806         <NA>       1.576819       3.698113  2024-09-09 06:53:10.909091072  2024-12-30 11:47:43.636363776                          982.549488                         982.549488           <NA>                        265.584815                       265.584815         1.636119    2024-09-11 16:41:26.380932352           94.875676                    9918.17603               9918.17603  2024-08-30 18:18:26.199460864  2024-09-12 18:19:59.999999744  2024-09-08 14:02:15.849056512                    NaN  2024-08-30 18:37:50.619945984  2024-09-08 14:02:15.849056512                       0.0               NaT             29.245283                        0.0            904.486146        471.348625           222316515.039548           0.0                            17.258760  2024-09-23 06:32:43.636363520           NaN          252.592480      158.146097                 NaN   
min              2024-01-05 10:14:16                          7.0             4.308125e+06              77550823.0         <NA>       1.000000            1.0            2024-01-03 00:00:00            2024-02-09 00:00:00                            0.000000                           0.000000           <NA>                          0.000000                         0.000000         0.000000              2024-01-05 10:14:16                89.0                         461.0                    461.0            2024-01-03 00:00:00            2024-01-05 00:00:00            2024-01-03 00:00:00                    NaN            2024-01-03 00:00:00            2024-01-03 00:00:00                       0.0               NaT             10.000000                        0.0              0.000000          0.000000                214483196.0           0.0                             0.000000            2024-01-29 00:00:00           NaN            0.000000        0.000000                 NaN   
25%    2024-05-14 02:21:21.249750016      

In [46]:
# obtendo o tipo de dados das colunas
data_types = df_la.dtypes

# obtendo a quantidade de valores nulos em cada coluna
null_values = df_la.isnull().sum()

#Combinand informações
info_colunas = pd.DataFrame({'Tipo de dados': data_types, 'Valores Nulos': null_values})

#visualizando as informações combinadas
print(info_colunas.to_string())


                                             Tipo de dados  Valores Nulos
Provisioning Hold Release Date              datetime64[ns]             17
Sold To IBM Customer Number                          Int64            104
Sold To Customer Number                              int64              0
Sold To Customer Name Full                          object              0
Sold To Country Code Description                    object              0
Geography                                           object              0
SAP Sales Order Number                               Int64              0
Distribution Channel                                object              0
Alternate Software Brand Code Dscr                  object              0
IBM Software Pillar Code                            object              0
Part Number                                          Int64            371
Revenue Stream Code Dscr                            object              0
IBM Software Pillar Code.1            

### Ainda tem algumas colunas com uma quantidade enorme de dados faltantes, mas vou prosseguir sem eles, acredito que não precisarei utilizar esses dados a principio. Vou criar algumas colunas novas e um dicionário para a nossa coluna de Industria, para no futuro plotar um grafico que mostre quais são as industrias que estão mais comprando na IBM e qual o tamanho dessas empresas.

In [ ]:
# Garantir que a coluna 'Sales Order Date' está no formato datetime
df_la['Sales Order Date'] = pd.to_datetime(df_la['Sales Order Date'])

# Criar a coluna 'Week' com o número da semana
df_la['Week'] = df_la['Sales Order Date'].dt.isocalendar().week

# Exibir o DataFrame com a nova coluna
#print(df_la)

In [63]:
df_2025 = df_la[df_la['Sales Order Date'].dt.year == 2025]

# Exibir as colunas 'Client', 'Sales Order Date' e 'Week' para o ano de 2025
print(df_2025[['Cliente', 'Sales Order Date', 'Week']])

                                Cliente Sales Order Date  Week
288              Randi Serviços Médicos       2025-01-02     1
289              Randi Serviços Médicos       2025-01-02     1
290              Randi Serviços Médicos       2025-01-02     1
291                               SEADE       2025-01-02     1
292                               SEADE       2025-01-02     1
..                                  ...              ...   ...
370                          Adrenorama       2025-04-30    18
371                          Adrenorama       2025-04-30    18
372  SCOPE INVESTIGACIÓN DE MERCADOS SC       2025-05-01    18
373  SCOPE INVESTIGACIÓN DE MERCADOS SC       2025-05-01    18
374                               ADASA       2025-05-02    18

[87 rows x 3 columns]


In [61]:
df_2025 = df_la[df_la['Sales Order Date'].dt.year == 2025]
# Exibir as colunas 'Client', 'Sales Order Date' e 'Week' para o ano de 2025
styled_df = df_2025[['Cliente', 'Sales Order Date', 'Week']].style.set_table_attributes('style="overflow:auto;max-height:300px;"')

# Mostrar o DataFrame com a rolagem
display(styled_df)

In [ ]:
industries_rev = {
    'Communication Sector': [
        'Inversiones Mabarod SA',
        'SPLIT SECOND RESEARCH BRASIL - LTDA',
        'OOL DIGITAL',
        'SA DISTRIB DE CONT AUDIOVISUAL LTDA',
        'HSR BRIDGE RESEARCH PESQ DE MERC LT',
        'Grifo Propaganda Ltda',
        'Casona Estúdio',
        'Conectare',
        'Cine Net SA de CV',
        'TIMES BRASIL MEDIA LTDA',
        'Samba do Alê',
        'LUMIO MARKETING E TEC DA INFO LTDA',
        'JULIANA CHRISTINE DIAS DA SILVA YAM',
        'CIRCULAR MEDIA LTDA',
        'ADRENORAMA',
        'Adrenorama'
    ],
    'Distribution Sector': [
        'BRASIL AGRO SERVICES LTDA',
        'LAURO MOLARI',
        'ALEXIS HELFON',
        'SIMA',
        'Particular',
        'DAVID BAHENA ESCALANTE',
        'Josue Ibañez Cruz',
        'Etelvina Robles Archudia',
        'Genesis LTDA',
        'Consultores Mercantiles SC',
        'UNIFORMES Y BORDADOS D"REYES',
        'Graciela Velazquez Ortega',
        'MBC SISTEMAS LTDA',
        'Carlos Juares Gehlen MEI',
        'Juan Manuel Calleja Castillo',
        'CONECTIVE',
        'Alejandro Padilla Isassi',
        'PRGM',
        'QVP Traduções e Serviços',
        'JUAN CARLOS BOCARANDO LARA',
        'Motel Corpo A Corpo',
        'TDSA COMERCIO DE SOFTWARE LTDA',
        'SCOPE INVESTIGACIÓN DE MERCADOS',
        'Redesign',
        'NORDSTERN TECHNOLOGIES, S.A. DE C.V',
        'On The Go',
        'Jonathan Arciga Dominguez',
        'Jonathan Arciga Dominguez',
        'RODONAVES TRANSPORTES E ENCOMENDAS',
        'VT Sistemas de Informática Ltda.',
        'Immersiva Lab',
        'RODHER GRUPO EMPRESARIAL DEL NORTE',
        'JIMENA CARDENAS MARQUEZ',
        'BEER4NOW',
        'Black Ecco TI S.A. de C.V.',
        'D Silveira Distribuidora',
        'ITPOWER SOFTWARE & SERVICOS LTDA',
        'Jane Doe',
        'Price Res, S.A.P.I. de C.V.',
        'GPLAN DESENVOLVIMENTO DE SOFTWARE L',
        'Data Harvest Solutions',
        'ACCESS & TI',
        'APMETTA',
        'MP Loyalty',
        'Alejandro Pérez Silva',
        'Sige Produciendo SA de CV',
        'Ana Beatriz C Sabra S Cestari Servi',
        'BULKY LOG TRANSPORTES LTDA',
        'Identa Consultores SC',
        'BIANCA XAVIER DE BRITO 37791764830',
        'SCOPE INVESTIGACIÓN DE MERCADOS SC',
        'Elda Frinne Galicia Moreno',
        'FRB SERVICOS DE PESQUISAS LTDA',
        'It Era',
        'NEUROVIM MEDICAL GROUP',
        'ILOS/LGSC - INSTITUTO DE LOGISTICA',
        'Jonathan Arciga Domínguez',
        'Senai Ist Automação',
        'HELIGA CONSULTORES',
        'Cloud Services & Training',
        'OFICINA DE VALOR',
        'QVP Soluções Linguisticas',
        'CONSULTORES MERCANTILES SC',
        'Distribuidora MazAuto del Noroeste',
        'DENTSPLY MEXICO, S.A. DE C.V.',
        'Gisela Ponce',
        'INSTITUTO MEXICANO DE',
        'ANNA CLARA SILVA DE ASSIS BARROS',
        'CTI Consultores Associados SS LTDA',
        'ESSELWARE SOLUCIONES',
        'HUGO HENRIQUE NASCIMENTO',
        'Intela',
        'Rhopen Consultoria LTDA'
    ],
    'Education': [
        'Parametria S. A. de C. V.',
        '53.510.836 SANDRA A F SEVILLA',
        'Genebaldo',
        'Profuturo Ltda',
        'UNIVERSIDADE DO ESTADO DA BAHIA',
        'Instituto Adventista de Ensino',
        'Universidad Autónoma Metropolitana',
        'UNAM',
        'Prix Projetos Mecanicos',
        'INSTITUTO TECNOLOGICO DE TRANSPORTE',
        'UFRRJ',
        'UESC/DCS',
        'Amanda Kovalczuk',
        'UFRJ',
        'IPADE',
        'INSTITUTO TECNOLOGICO Y DE ESTUDIOS',
        'Universidade Positivo',
        'Camino',
        'CCU JUSTO SIERRA',
        'Funcamp Funda√ß√£o Desenvolvimento Un',
        'Amanda Setti Raize',
        'UNIVERSIDAD AUTONOMA DE CHIHUAHUA',
        'Funcamp Fundação Desenvolvimento Un'
    ],
    'Financial Services': [
        'Uni Corretora',
        'Nakagawa Baptista & Baptista',
        'CGSTF',
        'WAGNER REGULADORA DE SINISTROS LTDA',
        'Trillon Services',
        'Guaná Agropecuária',
        'DESARROLLO EMPRESARIAL DEL LERMA SA DE CV',
        'NI For Ideas',
        'Medi Negocios Imobiliários Ltda',
        'GPI AUTO COMERCIAL SAPI DE C.V.',
        'OLIVARES PLATA CONSULTORES SA DE CV',
        'Lendia'
    ],
    'Health': [
        'PSEG CENTRO DE PESQUISA CLINICA',
        'GI Medicina Especializada',
        'addere medicina integrada',
        'Hospital de Clínicas de Porto Alegr',
        'Francisca Dominguez Dueñas',
        'CATARINE BENTA LOPES DOS SANTOS LTD',
        'Sergio Alfonso Patrón Chi',
        'NUNESMED LTDA',
        'hospital das clinicas da USP',
        'Clinica Roberta Moreira Allgayer LTDA',
        'DX GEN',
        'vascularline',
        'HEMATOLOGICA ALTA ESPECIALIDAD',
        'Dr Gabriel Eduardo Ornelas Cruz',
        'DKT DE MEXICO',
        'Ares Medicina Respiratoria',
        'Artem Curant',
        'Ana Karen Garro Almendaro',
        'Damiao e Infante Servicos Medicos S',
        'M2N',
        'Natália Marques Machado Ltda.',
        'Medicina Fetal México',
        'Toledo & Harada Servicos Medicos',
        'Randi Serviços Médicos',
        'Clinica Martins& Meister',
        'DNA Oncologia Eireli',
        'Inst. Monteiro de Neuromodulação',
        'BFG Cir Toracic',
        'CLINICA DE GERONI',
        'CLINICA AVANCADA EM RADIOTERAPIA E HEPATOLOGIA LTDA',
        'RADIOLOGIA E HEPATOLOGIA',
        'BFG Cir Toracica'
    ],
    'Industrial Sector': [
        'COMPAÑIA INTERNACIONAL DE COMERCIO',
        'Anvi Mexicana',
        'Gestión Integral de Proyectos de En',
        'HBR AVIACAO SA',
        'AMDA'
    ],
    'Public Sector': [
        'Auditoría Superior del Estado de Gu',
        'TRIBUNAL DE CONTAS DA UNIÃO',
        'SEADE',
        'Universidade Federal de Pernambuco',
        'UN KILO DE AYUDA',
        'FAPERJ - FUNDACAO DE AMPARO A',
        'ADASA'
    ]
}

# Converter o dicionário em um DataFrame
df_industries = pd.DataFrame([(cliente, setor) for setor, clientes in industries_rev.items() for cliente in clientes], columns=['Cliente', 'Setor'])

# Visualizar as primeiras linhas do novo DataFrame
df_industries.head()


Cliente                 Setor
0               Inversiones Mabarod SA  Communication Sector
1  SPLIT SECOND RESEARCH BRASIL - LTDA  Communication Sector
2                          OOL DIGITAL  Communication Sector
3  SA DISTRIB DE CONT AUDIOVISUAL LTDA  Communication Sector
4  HSR BRIDGE RESEARCH PESQ DE MERC LT  Communication Sector

In [48]:
df_la = df_la.merge(df_industries, left_on='Sold To Customer Name Full', right_on='Cliente', how='left')

# Visualizar as primeiras linhas do novo DataFrame
#df_la.head()


In [ ]:
# Visualizar todas as colunas específicas
#print(df_la[['Sold To Customer Name Full', 'IBM Software Brand Code', 'Cliente', 'Setor']].to_string())

In [49]:
# Calcular a diferença em dias entre 'End Date' e 'Start Date'
df_la['Dias Ativos'] = (df_la['End Date'] - df_la['Start Date']).dt.days
# Visualizar as primeiras linhas do DataFrame com a nova coluna
print(df_la[['Sold To Customer Name Full', 'Start Date', 'End Date', 'Dias Ativos']].to_string())

                              Sold To Customer Name Full Start Date   End Date  Dias Ativos
0                              BRASIL AGRO SERVICES LTDA 2024-01-03 2025-01-02        365.0
1                              BRASIL AGRO SERVICES LTDA 2024-01-03 2025-01-02        365.0
2                             Elda Frinne Galicia Moreno 2024-01-10 2024-02-09         30.0
3                        PSEG CENTRO DE PESQUISA CLINICA 2024-01-12 2024-02-11         30.0
4                     LUMIO MARKETING E TEC DA INFO LTDA 2024-01-17 2024-02-16         30.0
5                     LUMIO MARKETING E TEC DA INFO LTDA 2024-01-17 2024-02-16         30.0
6                         FRB SERVICOS DE PESQUISAS LTDA 2024-01-18 2024-02-17         30.0
7                    INSTITUTO TECNOLOGICO Y DE ESTUDIOS 2024-01-23 2025-01-22        365.0
8                    INSTITUTO TECNOLOGICO Y DE ESTUDIOS 2024-01-23 2025-01-22        365.0
9                                 Inversiones Mabarod SA 2024-01-24 2024-02-23  

In [50]:
# Criar uma nova coluna com o dia da semana
df_la['Dia da Semana'] = df_la['Start Date'].dt.day_name()

# Visualizar as primeiras linhas do DataFrame com a nova coluna
print(df_la[['Sold To Customer Name Full', 'Start Date', 'Dia da Semana']].head())


           Sold To Customer Name Full Start Date Dia da Semana
0           BRASIL AGRO SERVICES LTDA 2024-01-03     Wednesday
1           BRASIL AGRO SERVICES LTDA 2024-01-03     Wednesday
2          Elda Frinne Galicia Moreno 2024-01-10     Wednesday
3     PSEG CENTRO DE PESQUISA CLINICA 2024-01-12        Friday
4  LUMIO MARKETING E TEC DA INFO LTDA 2024-01-17     Wednesday


In [51]:
# Definir as prioridades
prioridade_1 = ['SPSS Statistics', 'Aspera', 'Cognos Analytics']
prioridade_2 = ['Blueworks Live']

# Função para determinar a prioridade
def determinar_prioridade(brand_code):
    if brand_code in prioridade_1:
        return 'Prioridade 1'
    elif brand_code in prioridade_2:
        return 'Prioridade 2'
    else:
        return 'Prioridade 3'

# Aplicar a função para criar a nova coluna 'Prioridade'
df_la['Prioridade'] = df_la['Product Name'].apply(determinar_prioridade)

# Visualizar as primeiras linhas do DataFrame com a nova coluna
print(df_la[['Sold To Customer Name Full', 'Product Name', 'Prioridade']].head())

           Sold To Customer Name Full     Product Name    Prioridade
0           BRASIL AGRO SERVICES LTDA          MaaS360  Prioridade 3
1           BRASIL AGRO SERVICES LTDA          MaaS360  Prioridade 3
2          Elda Frinne Galicia Moreno  SPSS Statistics  Prioridade 1
3     PSEG CENTRO DE PESQUISA CLINICA  SPSS Statistics  Prioridade 1
4  LUMIO MARKETING E TEC DA INFO LTDA  SPSS Statistics  Prioridade 1


In [52]:
df_la['Unique'] = df_la.duplicated(subset=['Sold To Customer Name Full'], keep='first').apply(lambda x: 'n' if x else 'y')

In [ ]:
#df_la.info()

In [53]:


unique_y_count = df_la[df_la['Unique'] == 'y'].shape[0]
print(f'A quantidade de clientes únicos no dataset é: {unique_y_count}.')


A quantidade de clientes únicos no dataset é: 161.


In [54]:

# Definição do intervalo de datas
inicio = pd.to_datetime('2025-03-01')
fim = pd.to_datetime('2025-05-21')

# Filtro das transações dentro do intervalo e com valor diferente de zero
transacoes_filtradas = df_la[
    (df_la['Sales Order Date'] >= inicio) & 
    (df_la['Sales Order Date'] <= fim) &
    (df_la['SaaS Total Commitment Value USD'] != 0)
    #(df_la['Cancel Reason Dscr'] == '')

]

# Seleção das colunas desejadas
colunas_desejadas = [
    'Sold To Customer Name Full',
    'Sold To IBM Domestic Client Number',
    'Sold To Country Code Description',
    'Sales Order Date',
    'Hold status new',
    'Cancel Reason Dscr'
]

transacoes_filtradas = transacoes_filtradas[colunas_desejadas]

# Renomeando as colunas
transacoes_filtradas = transacoes_filtradas.rename(columns={
    'Sold To Customer Name Full': 'Client',
    'Sold To IBM Domestic Client Number': 'IBM Number',
    'Sold To Country Code Description': 'Country',
    'Sales Order Date': 'Order Date',
    'Hold status new': 'Hold status',
    'Cancel Reason Dscr': 'Cancel Reason'
})
#

# Agrupamento por cliente e contagem de transações
transacoes_agrupadas = (
    transacoes_filtradas
    .groupby(['Client', 'Country', 'Order Date', 'Hold status', 'Cancel Reason'])
    .size()
    .reset_index(name='Frequência')
)

# Exibição do resultado
print(transacoes_agrupadas)



                             Client Country Order Date Hold status                 Cancel Reason  Frequência
0                  CCU JUSTO SIERRA  Mexico 2025-03-12     On Hold        SaaS Early Termination           1
1        CONSULTORES MERCANTILES SC  Mexico 2025-03-10     On Hold        SaaS Early Termination           2
2           Rhopen Consultoria LTDA  Brazil 2025-03-31    Released  SaaS End of Term Termination           1
3  Toledo & Harada Servicos Medicos  Brazil 2025-03-03    Released  SaaS End of Term Termination           1


In [55]:

total_clientes_unicos = transacoes_filtradas['Client'].nunique()

# Adiciona uma linha com o total de clientes únicos
total_clientes_df = pd.DataFrame([['Total de Clientes Únicos', '', '', total_clientes_unicos, '', '']], columns=transacoes_agrupadas.columns)

# Concatena ao DataFrame agrupado
transacoes_agrupadas = pd.concat([transacoes_agrupadas, total_clientes_df], ignore_index=True)

# Exibição do resultado
print(transacoes_agrupadas)


                             Client Country           Order Date Hold status                 Cancel Reason Frequência
0                  CCU JUSTO SIERRA  Mexico  2025-03-12 00:00:00     On Hold        SaaS Early Termination          1
1        CONSULTORES MERCANTILES SC  Mexico  2025-03-10 00:00:00     On Hold        SaaS Early Termination          2
2           Rhopen Consultoria LTDA  Brazil  2025-03-31 00:00:00    Released  SaaS End of Term Termination          1
3  Toledo & Harada Servicos Medicos  Brazil  2025-03-03 00:00:00    Released  SaaS End of Term Termination          1
4          Total de Clientes Únicos                                       25                                         


In [56]:

# Obter a data atual no formato YYYY-MM-DD
current_date = datetime.now().strftime('%Y-%m-%d')

# Especificar o caminho da pasta onde o arquivo será salvo
folder_cd_path = '/Users/jhenyfferborges/Documents/DSW_analysis/projeto_dsw/clean_data'

# Criar o nome do arquivo com a data atual
df_la_clean = f'df_la_{current_date}.csv'

# Criar o caminho completo do arquivo
file_path = os.path.join(folder_cd_path, df_la_clean)

# Salvar o DataFrame em um arquivo CSV com a data atual no nome do arquivo
df_la.to_csv(file_path, index=False)


### Terminada a parte de limpeza dos dados, agora finalmente iremos entrar na parte de visualizar os dados. Vamos brincar com graficos